In [2]:
#Importing libraries
#importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('punkt')
import unicodedata 
#from contractions import CONTRACTION_MAP # from contractions.py
import contractions
import re
import nltk
import string 
import spacy
from nltk.tokenize import ToktokTokenizer
from nltk.tokenize import word_tokenize
from pickle import dump
from collections import Counter
from nltk import ngrams
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils
import pickle
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

from keras.models import Sequential
from keras.layers import Dense
from keras.metrics import Precision
from keras.metrics import Recall
from keras.metrics import Accuracy
from keras.metrics import AUC
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from numpy import mean
from numpy import std
import ast
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

In [3]:

df = pd.read_csv("openLDF.csv");
df.head()

,Unnamed: 0,id,Atelectasis,Cardiomegaly,Effusion,Infiltration,Mass,Nodule,Pneumonia,Pneumothorax,Consolidation,Edema,Emphysema,Fibrosis,Pleural_Thickening,Hernia,Normal,split,TXT
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,train,The cardiac silhouette and mediastinum size ar...
1,1021,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,train,Borderline cardiomegaly. Midline sternotomy XX...
2,2020,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,train,"No displaced rib fractures, pneumothorax, or ..."
3,3061,4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,train,There are diffuse bilateral interstitial and a...
4,3169,5,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,train,The cardiomediastinal silhouette and pulmonary...


In [ ]:
#Merge all labels to a list
df['labels']= df[['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax', 'Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia', 'Normal']].values.tolist()
len(df['labels'][0])

15

In [ ]:
#Drop all individual columns
df
#df.drop(['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax'], axis=1, inplace=True)

,Unnamed: 0,id,Atelectasis,Cardiomegaly,Effusion,Infiltration,Mass,Nodule,Pneumonia,Pneumothorax,Consolidation,Edema,Emphysema,Fibrosis,Pleural_Thickening,Hernia,Normal,split,TXT,labels
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,train,The cardiac silhouette and mediastinum size ar...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
1,1021,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,train,Borderline cardiomegaly. Midline sternotomy XX...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,2020,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,train,"No displaced rib fractures, pneumothorax, or ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
3,3061,4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,train,There are diffuse bilateral interstitial and a...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
4,3169,5,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,train,The cardiomediastinal silhouette and pulmonary...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3679,3056,3995,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,train,The cardiomediastinal silhouette and pulmonary...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
3680,3057,3996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,train,The lungs are clear. Heart size is normal. No ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
3681,3058,3997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,train,"Heart size within normal limits. Small, nodula...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
3682,3059,3998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,train,Heart size is normal and the lungs are clear.,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"


In [ ]:
df["TXT"] = df["TXT"].astype("string")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3684 entries, 0 to 3683
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          3684 non-null   int64 
 1   id                  3684 non-null   int64 
 2   Atelectasis         3684 non-null   int64 
 3   Cardiomegaly        3684 non-null   int64 
 4   Effusion            3684 non-null   int64 
 5   Infiltration        3684 non-null   int64 
 6   Mass                3684 non-null   int64 
 7   Nodule              3684 non-null   int64 
 8   Pneumonia           3684 non-null   int64 
 9   Pneumothorax        3684 non-null   int64 
 10  Consolidation       3684 non-null   int64 
 11  Edema               3684 non-null   int64 
 12  Emphysema           3684 non-null   int64 
 13  Fibrosis            3684 non-null   int64 
 14  Pleural_Thickening  3684 non-null   int64 
 15  Hernia              3684 non-null   int64 
 16  Normal              3684

In [ ]:
#Splitting Training and testing
train = df[df["split"] == "train"]
test = df[df["split"] == "test"]
#train, test = train_test_split(df, test_size=0.3, random_state=42)


In [ ]:
##Tokenise the sentences 
from nltk.corpus import stopwords
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens
train_tagged = train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['TXT']), tags=r.labels), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['TXT']), tags=r.labels), axis=1)

In [ ]:
train_tagged.values[0]

TaggedDocument(words=['the', 'cardiac', 'silhouette', 'and', 'mediastinum', 'size', 'are', 'within', 'normal', 'limits', 'there', 'is', 'no', 'pulmonary', 'edema', 'there', 'is', 'no', 'focal', 'consolidation', 'there', 'are', 'no', 'xxxx', 'of', 'pleural', 'effusion', 'there', 'is', 'no', 'evidence', 'of', 'pneumothorax', 'normal', 'chest', 'x-xxxx'], tags=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [ ]:
#Creating a doc2vec model
cores = multiprocessing.cpu_count()
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)]) #both words and tags to doc2Vec

100%|██████████| 2912/2912 [00:00<00:00, 4158601.72it/s]


In [ ]:
#Train doc2vec model
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 2912/2912 [00:00<00:00, 4708486.22it/s]


In [ ]:
# save the model to disk
filename = '../DataSet/CSV/doc2vec_dbow_model.sav'
pickle.dump(model_dbow, open(filename, 'wb'))

In [ ]:
# #Load file
with open('../DataSet/CSV/doc2vec_dbow_model.sav', 'rb') as handle:
    model_dbow = pickle.load(handle)

In [ ]:
#Creating the feature vectors for each paragraphs 
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags, model.infer_vector(doc.words, alpha=0.1, min_alpha=0.005, epochs=100)) for doc in sents])
    return targets, regressors
y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)

In [ ]:
#print((X_train[0]))
#print(y_train[0])
#X_train[0].shape
#y_train[0].shape
print(len(X_train[0]))
print(len(y_train[0]))
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)


300
15


# Neural Network model

In [ ]:
n_input = 300
n_output = 15

model = Sequential()

#input layer
model.add(Dense(64, activation='relu', input_dim=n_input))

#hidden layer
model.add(Dense(32, activation='relu'))

#output layer
model.add(Dense(n_output, activation='softmax'))

#compiling the model with loss function binary cross entropy
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[Precision(), Recall(), Accuracy(), AUC()])

#train the model
model.fit(X_train, y_train, epochs=100, batch_size=100)

Epoch 1/100
30/30 [==============================] - 1s 3ms/step - loss: 0.6336 - precision_3: 0.2724 - recall_3: 0.0160 - accuracy: 0.0000e+00 - auc_3: 0.6511    
Epoch 2/100
30/30 [==============================] - 0s 2ms/step - loss: 0.1554 - precision_3: 0.8035 - recall_3: 0.6864 - accuracy: 0.0000e+00 - auc_3: 0.9193
Epoch 3/100
30/30 [==============================] - 0s 2ms/step - loss: 0.1082 - precision_3: 0.8129 - recall_3: 0.6992 - accuracy: 0.0000e+00 - auc_3: 0.9488
Epoch 4/100
30/30 [==============================] - 0s 3ms/step - loss: 0.0914 - precision_3: 0.8692 - recall_3: 0.7082 - accuracy: 0.0000e+00 - auc_3: 0.9661
Epoch 5/100
30/30 [==============================] - 0s 3ms/step - loss: 0.0824 - precision_3: 0.8724 - recall_3: 0.7400 - accuracy: 0.0000e+00 - auc_3: 0.9685
Epoch 6/100
30/30 [==============================] - 0s 3ms/step - loss: 0.0760 - precision_3: 0.8915 - recall_3: 0.7580 - accuracy: 0.0000e+00 - auc_3: 0.9764
Epoch 7/100
30/30 [=================

## Model prediction, Evaluation

In [ ]:
#test on unseen data
y_pred = model.predict(X_test)
y_pred = y_pred.round()
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average=None)
recall = recall_score(y_test, y_pred, average=None)

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# summarize performance
print('Accuracy: %.3f' % (mean(accuracy)))
print('Precision: %.3f' % (mean(precision)))
print('Recall: %.3f' % (mean(recall)))

Accuracy: 0.784
Precision: 0.588
Recall: 0.326


In [ ]:
final_list = []

for index, item in enumerate(y_test):
    set_1 = [item,y_pred[index]]
    final_list.append(set_1)
df_final = pd.DataFrame(final_list, columns = ['actual', 'Doc2Vec_prediction'])

In [ ]:
df_final

,actual,Doc2Vec_prediction
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
767,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
768,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
769,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
770,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
df_final.to_csv('Doc2Vec_actual_prediction.csv')

In [ ]:
import pandas as pd
df = pd.read_csv('Doc2Vec_actual_prediction.csv', sep='\t')
df

,Unnamed: 0,actual,Doc2Vec_prediction
0,0,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1],[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
1,1,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1],[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
2,2,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1],[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
3,3,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1],[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
4,4,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1],[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
...,...,...,...
767,767,[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0],[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
768,768,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1],[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
769,769,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1],[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
770,770,[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1],[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
